<a href="https://colab.research.google.com/github/tthandi/magic_formula_project/blob/main/AlphaVantageDataLimitationsTests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test of daily adjusted price data for popular tickers


In [ ]:
# ----------------------------------------------------------------------
# 0. Install libraries, import modules, authenticate to GCP
# ----------------------------------------------------------------------
!pip install --quiet --upgrade pandas google-cloud-storage google-cloud-bigquery

import io, json, requests, pandas as pd
from google.colab import auth
from google.cloud import storage, bigquery
from google.cloud.exceptions import NotFound

auth.authenticate_user()

# ----------------------------------------------------------------------
# 1. Configuration
# ----------------------------------------------------------------------
PROJECT_ID  = ""
DATASET_ID  = ""
BUCKET_NAME = ""
LOCATION    = ""
API_KEY     = ""
TICKERS     = ["AAPL", "MSFT", "IBM", "GE"]
YEARS       = ["2024", "2014", "2004", "1994", "1984"]

# ----------------------------------------------------------------------
# 2. Get or create GCS bucket and BigQuery dataset
# ----------------------------------------------------------------------
storage_client = storage.Client(project=PROJECT_ID)

try:
    bucket = storage_client.get_bucket(BUCKET_NAME)
    print("✅ Reusing bucket:", BUCKET_NAME)
except NotFound:
    bucket = storage_client.bucket(BUCKET_NAME)
    bucket.storage_class = "STANDARD"
    storage_client.create_bucket(bucket, location=LOCATION)
    print("✅ Created bucket:", BUCKET_NAME)

bq_client = bigquery.Client(project=PROJECT_ID)
try:
    bq_client.get_dataset(DATASET_ID)
    print("✅ BigQuery dataset exists:", DATASET_ID)
except NotFound:
    ds = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
    ds.location = LOCATION
    bq_client.create_dataset(ds)
    print("✅ Created dataset:", DATASET_ID)

# ----------------------------------------------------------------------
# 3. Helper functions
# ----------------------------------------------------------------------
def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    return df.rename(columns=lambda c: str(c).lower()
                                   .strip().replace(" ", "_").replace(".", ""))

def write_gcs(df: pd.DataFrame, blob_name: str) -> None:
    path = f"/tmp/{blob_name}"
    df.to_csv(path, index=False, encoding="utf-8")
    bucket.blob(blob_name).upload_from_filename(path)
    print("📤", f"gs://{BUCKET_NAME}/{blob_name}")

def load_bq(clean_blob: str) -> None:
    table_id = f"{PROJECT_ID}.{DATASET_ID}.{clean_blob[:-4].lower()}"
    cfg = bigquery.LoadJobConfig(
        autodetect=True,
        skip_leading_rows=1,
        source_format=bigquery.SourceFormat.CSV,
        write_disposition=bigquery.job.WriteDisposition.WRITE_TRUNCATE
    )
    uri = f"gs://{BUCKET_NAME}/{clean_blob}"
    bq_client.load_table_from_uri(uri, table_id, job_config=cfg).result()
    print("✅ BigQuery:", table_id)

def fetch_listing_status(state="active") -> pd.DataFrame:
    r = requests.get(
        "https://www.alphavantage.co/query",
        params={
            "function": "LISTING_STATUS",
            "apikey"  : API_KEY,
            "state"   : state,
            "datatype": "csv"
        },
        timeout=15
    )
    print("🔎", r.url, "→", r.status_code, len(r.text), "bytes")
    r.raise_for_status()
    return pd.read_csv(io.StringIO(r.text)) if r.text.strip() else pd.DataFrame()

def fetch_json(fn, symbol=None, **extra) -> dict:
    base = {"function": fn, "apikey": API_KEY, **extra}
    if symbol:
        base["symbol"] = symbol
    r = requests.get("https://www.alphavantage.co/query", params=base, timeout=15)
    print("🔎", r.url, "→", r.status_code)
    r.raise_for_status()
    try:
        data = r.json()
    except json.JSONDecodeError:
        print("⚠️ non-JSON response")
        return {}
    if "Note" in data:
        print("⚠️", data["Note"][:120])
    return data

# ----------------------------------------------------------------------
# 4. LISTING_STATUS (active & delisted) – run once
# ----------------------------------------------------------------------
for state in ("active", "delisted"):
    df_ls = fetch_listing_status(state)
    if not df_ls.empty:
        write_gcs(df_ls,                 f"listing_status_{state}_raw.csv")
        cleaned = clean_cols(df_ls)
        clean_name = f"listing_status_{state}.csv"
        write_gcs(cleaned, clean_name)
        load_bq(clean_name)

# ----------------------------------------------------------------------
# 5. Ticker × Year loop for daily-adjusted data
# ----------------------------------------------------------------------
for tic in TICKERS:
    print(f"\n📈 {tic} — daily-adjusted history slices")
    full_json = fetch_json("TIME_SERIES_DAILY_ADJUSTED", tic, outputsize="full")
    all_daily = full_json.get("Time Series (Daily)", {})

    for yr in YEARS:
        rows = {d: v for d, v in all_daily.items() if d.startswith(yr)}
        if not rows:
            print(f"ℹ️ {tic} has no data for {yr} — skipped")
            continue

        df = (
            pd.DataFrame.from_dict(rows, orient="index")
              .reset_index()
              .rename(columns={"index": "date"})
        )

        raw_name   = f"{tic}_daily_adjusted_{yr}_raw.csv"
        clean_name = f"{tic}_daily_adjusted_{yr}.csv"
        write_gcs(df, raw_name)
        write_gcs(clean_cols(df), clean_name)
        load_bq(clean_name)

print("\n Finished — raw CSVs in gs://{} and cleaned tables in BigQuery.".format(BUCKET_NAME))


Test for adjusted price data for popular and delisted tickers


In [ ]:
#!/usr/bin/env python3
"""
av_history_depth_plus.py  (revised – no CIK assumption)
"""

import os, io, csv, json, time, requests, pandas as pd
from datetime import date

API_KEY = os.getenv("AV_API_KEY")
if not API_KEY:
    raise SystemExit("export AV_API_KEY first")

BASE  = "https://www.alphavantage.co/query"
SLEEP = 0.8    # throttle ~75 req/min

# ── ticker universe (active + delisted sample) ─────────────────────────
LONG_LIVED = {
    "IBM": "International Business Machines",
    "GE":  "General Electric",
    "KO":  "Coca-Cola Company",
    "PG":  "Procter & Gamble",
}
DELISTED = {
    "USG":   "USG Corp – private 2019",
    "DWA":   "DreamWorks – bought 2016",
    "SKUL":  "Skullcandy – private 2016",
    "SYX":   "Systemax – ticker now GIC",
    "KRFT":  "Kraft Foods Group – merged 2015",
    "LEHMQ": "Lehman Brothers – bankruptcy 2008",
    "ENRNQ": "Enron – bankruptcy 2001",
}
TICKERS = {**LONG_LIVED, **DELISTED}

ENDPOINTS = {
    "DAILY":   {"function": "TIME_SERIES_DAILY_ADJUSTED",   "outputsize": "full"},
    "WEEKLY":  {"function": "TIME_SERIES_WEEKLY_ADJUSTED"},
    "MONTHLY": {"function": "TIME_SERIES_MONTHLY_ADJUSTED"},
}

# ── download listing snapshots once ────────────────────────────────────
def fetch_listing(state: str) -> pd.DataFrame:
    r = requests.get(BASE, params={
        "function": "LISTING_STATUS",
        "state": state,
        "datatype": "csv",
        "apikey": API_KEY}, timeout=20)
    r.raise_for_status()
    df = pd.read_csv(io.StringIO(r.text))
    df.columns = [c.lower() for c in df.columns]      # normalise
    return df

print("Downloading LISTING_STATUS snapshots …")
active_df   = fetch_listing("active")
delisted_df = fetch_listing("delisted")
active_set   = set(active_df["symbol"])
delisted_set = set(delisted_df["symbol"])

# (optional) build successor map here if you have manual aliases
SUCCESSOR_MAP = {
    # "SYX": "GIC",   # example
}

# ── simple resolver (no CIK fallback) ───────────────────────────────────
def resolve_symbol(sym: str) -> str:
    sym_clean = sym.rstrip("QPK")           # drop OTC suffixes
    return SUCCESSOR_MAP.get(sym_clean, sym_clean)

# ── fetch one series helper ─────────────────────────────────────────────
def fetch_series(sym: str, params: dict) -> tuple[pd.DataFrame, str]:
    r = requests.get(BASE, params=params | {"symbol": sym, "apikey": API_KEY})
    if r.status_code != 200:
        return pd.DataFrame(), f"http {r.status_code}"
    try:
        j = r.json()
    except json.JSONDecodeError:
        return pd.DataFrame(), "non-json"
    if "Note" in j:           return pd.DataFrame(), "rate-limit"
    if "Error Message" in j:  return pd.DataFrame(), "api-error"
    key = next((k for k in j if "Series" in k), None)
    if not (key and j[key]):  return pd.DataFrame(), "empty"
    df = (pd.DataFrame.from_dict(j[key], orient="index")
            .reset_index().rename(columns={"index":"date"}))
    return df, "ok"

# ── main probe loop ─────────────────────────────────────────────────────
records = []
for orig, note in TICKERS.items():
    sym = resolve_symbol(orig)
    rec = {"symbol": orig, "resolved": sym, "note": note,
           "in_active": sym in active_set, "in_delisted": sym in delisted_set}
    earliest = latest = rows = None

    for tag, ep in ENDPOINTS.items():
        df, reason = fetch_series(sym, ep)
        time.sleep(SLEEP)
        rec[f"{tag.lower()}_status"] = "✅" if not df.empty else f"❌ ({reason})"

        if earliest is None and not df.empty:          # first successful series
            earliest = df["date"].min()
            latest   = df["date"].max()
            rows     = len(df)
            df.to_csv(f"{orig}_{tag.lower()}_alpha.csv", index=False)

    rec.update({"earliest": earliest or "n/a",
                "latest"  : latest   or "n/a",
                "rows"    : rows     or 0})
    records.append(rec)

# ── summary table ───────────────────────────────────────────────────────
cols = ["symbol","resolved","note","earliest","latest","rows",
        "daily_status","weekly_status","monthly_status","in_delisted","in_active"]
df_out = pd.DataFrame(records)[cols]

print(f"\nAlpha Vantage history depth — {date.today()}\n")
print(df_out.to_markdown(index=False))
